In [125]:
import matplotlib.pyplot as plt
import numpy as np

from queue import PriorityQueue

from IPython.display import clear_output
import time

In [126]:
data_path = 'data3.txt' # -data
max_cluster_num = 20 # -k
min_cluster_size = 5 # -s
min_intra_dist = 0.9 # -d
output_path = 'output3.txt' # -output

MAGIC_NUMBER = 100

In [127]:
data = np.genfromtxt(data_path, delimiter=' ')

DATA_SIZE = data.shape[0]
DATA_DIM = data.shape[1]

feature_idx = np.array([i for i in range(DATA_SIZE)], dtype=int)

# print(data[:5])
# print(data.shape)

In [128]:
label = {i: 0 for i in range(DATA_SIZE)}

In [129]:
# # Compute the pair-wise distance matrix

# # Initialize the matrix
# distance_mat = np.zeros((DATA_SIZE, DATA_SIZE))

# # Fill in with true distance
# for i in range(DATA_SIZE):
#     for j in range(DATA_SIZE):
        
#         # dist(a,a) = 0
#         if i == j:
#             continue
            
#         # dist(a,b) = dist(b,a)
#         if i > j:
#             distance_mat[i,j] = distance_mat[j,i]
            
#         # Calculate the Euclidean distance: dist(a,b)
#         else:
#             distance_mat[i,j] = np.linalg.norm(data[i] - data[j])

# plt.matshow(distance_mat, cmap='gray')
# plt.show()

In [130]:
def dist(a, b):
    return np.linalg.norm(a - b)

In [131]:
def icd(features, center):

    intra_dist = np.array([])

    for f in features:
        
        distance = dist(center, f, False)
        intra_dist = np.append(intra_dist, distance)

    return np.mean(intra_dist)

In [132]:
def kmeans(feature_idx, k=2):

    feature_dim = DATA_DIM
    
    # Initialize K centers
    center_idx = np.random.choice(feature_idx, size=k, replace=False)
    center = np.array(data[center_idx])
    cluster_id = [i for i in range(k)]

    # Initialize return values
    width = [] # intra-cluster distance
    clusters = []

    while True:

        # Assign each x to its closest center
        label = np.array([], dtype=int)
        dist2center = np.array([])

        for f in feature_idx:

            min_distance = None
            nearest_label = None

            for i in range(k):

                c = center[i]
                distance = dist(data[f], c)

                # Update to the closest center so far
                if min_distance is None or distance < min_distance:  # Short circuit
                    
                    min_distance = distance
                    nearest_label = cluster_id[i]
            
            dist2center= np.append(dist2center, min_distance)
            label = np.append(label, nearest_label)

        # Calculate the mean of each cluster
        cluster_mean = np.array([])

        for lab in cluster_id:

            cluster = np.array([data[i] for i in feature_idx[label == lab]])
            cluster_mean = np.append(cluster_mean, np.mean(cluster, axis=0))

        cluster_mean = np.reshape(cluster_mean, (-1, feature_dim))

        # Continue updating the cluster centers if any x is reassigned to a new center
        if np.array_equal(cluster_mean, center):
            
            # Output block:
            # Calculate the width of each cluster
            bitmap = []
            
            for i, lab in zip(range(k), cluster_id):
                
                idx_position = label == lab
                bitmap.append((sum(idx_position), idx_position))
            
            bitmap.sort(key=lambda x : x[0])
            
            for (_, idx_position) in bitmap:
                
                width.append(np.mean(dist2center[idx_position]))

                clusters.append(feature_idx[idx_position])
            
            break

        else:
            
            center = cluster_mean

    return width, clusters

In [133]:
# kmeans(feature_idx)

In [134]:
class Node:
    
    node_queue = PriorityQueue()
    priority_set = set()
    
    def __init__(self, feature_index, width=MAGIC_NUMBER):
        
        # Data and children
        self.data = feature_index
        self.left = None
        self.right = None
        
        # Attributes
        self.size = len(feature_index)
        self.width = width
        self.isLeaf = False
        self.end = False
        
        self.enqueue(offset=self.size)


    def clear_node_queue():
        
        Node.node_queue = PriorityQueue()
    
    def dequeue():
        
        return Node.node_queue.get()[1]
    
    def qsize():
        
        return Node.node_queue.qsize()
    
    def queue_is_empty():
        
        return Node.node_queue.empty()
    
    def enqueue(self, offset=0):
        
        priority = Node.generate_priority(DATA_SIZE - offset)
        Node.node_queue.put((priority, self))
        Node.priority_set.add(priority)
    
    def generate_priority(priority):
        
        if not {priority}.issubset(Node.priority_set):
        
            return priority
        
        else:
            
            return Node.generate_priority(priority + 1 / DATA_SIZE)
        

In [135]:
def bisecting(recursive=True):
    
    node = Node.dequeue()
    
    if node.isLeaf:
        
        node.enqueue()
        return True # Stop
    
    if node.width < min_intra_dist:
        
        node.isLeaf = True
        node.enqueue()
        
        return bisecting() if recursive else False # Continue to next node
    
    if node.size < min_cluster_size:
        
        node.enqueue()
        return True # Stop
    
    [left_width, right_width], [left_cluster, right_cluster] = kmeans(node.data)
    
    node.left = Node(left_cluster, left_width)
    node.right = Node(right_cluster, right_width)
    
    if Node.qsize() >= max_cluster_num:
        
        return True # Stop
    
    return bisecting() if recursive else False # Continue to next node

In [136]:
# stop = bisecting(False)

# while not stop:
    
#     clear_output(wait=True)
#     print(Node.qsize())
#     print_dendrogram(root)
#     time.sleep(0.2)
#     stop = bisecting(False)

In [137]:
class store:
    
    # Storage
    storage = {}

    def emit(depth, data):
        current = store.storage.get(depth, [])
        current.append(str(data))
        store.storage[depth] = [i for i in current]
        
    def print_tree():
        for k in range(len(store.storage)):
            if store.storage[k]:
                line = ' '.join(store.storage[k])
                print(line)


In [138]:
def print_dendrogram(node, depth=0):
    
    line = ''
    padding = depth
    
    if not node:
        return
    
    store.emit(depth, node.size)
    
    print_dendrogram(node.left, padding+1)
    
    print_dendrogram(node.right, padding+1)
    
    return

In [139]:
def print_dendrogram_2(node, depth=1, pos=[1], right_tree=True, print_icd=False):
    
    line = ''
    padding = depth
    position = [pos[i] for i in range(len(pos))]
    
    if not node:
        return
    
    for i in range(padding-1):
        if position[i] != 1:
            line += '│     '
        else:
            line += '      '
    
    if right_tree:
        line += '└──── '
    else:
        line += '├──── '
        
    
    line += str(node.size)
    
    if not node.left or not node.right:
        
        leaf = f'icd = {node.width:.3f}' if print_icd else ''
        line += ': Leaf ' + leaf
    
    print(line)
    
    position_l = [pos[i] for i in range(len(pos))]
    position_l.append(0)

    print_dendrogram_2(node.left, padding+1, position_l, False, print_icd)
    
    position_r = [pos[i] for i in range(len(pos))]
    position_r.append(1)

    print_dendrogram_2(node.right, padding+1, position_r, True, print_icd)

In [140]:
Node.clear_node_queue()
root = Node(feature_idx)

print(bisecting())

True


In [141]:
# Basic
store.storage = {}
print_dendrogram(root)
store.print_tree()

1000
492 508
171 321 212 296
72 99 154 167 48 164 47 249
32 40 36 63 65 89 72 95 72 92 51 198
44 45 16 56 45 50 38 54 55 143
43 100


In [142]:
# Better looking
print(Node.qsize(), 'leaves')
print()

print_dendrogram_2(root)

20 leaves

└──── 1000
      ├──── 492
      │     ├──── 171
      │     │     ├──── 72
      │     │     │     ├──── 32: Leaf 
      │     │     │     └──── 40: Leaf 
      │     │     └──── 99
      │     │           ├──── 36: Leaf 
      │     │           └──── 63: Leaf 
      │     └──── 321
      │           ├──── 154
      │           │     ├──── 65: Leaf 
      │           │     └──── 89
      │           │           ├──── 44: Leaf 
      │           │           └──── 45: Leaf 
      │           └──── 167
      │                 ├──── 72
      │                 │     ├──── 16: Leaf 
      │                 │     └──── 56: Leaf 
      │                 └──── 95
      │                       ├──── 45: Leaf 
      │                       └──── 50: Leaf 
      └──── 508
            ├──── 212
            │     ├──── 48: Leaf 
            │     └──── 164
            │           ├──── 72: Leaf 
            │           └──── 92
            │                 ├──── 38: Leaf 
            │ 

In [143]:
def print_clusters(node, leaf_visited):
    
    if node.isLeaf:
        
        for i in node.data:
            
            label[i] = leaf_visited
        
        return leaf_visited + 1
    
    if node.left:
        
        leaf_visited = print_clusters(node.left, leaf_visited)
    
    if node.right:
        
        leaf_visited = print_clusters(node.right, leaf_visited)
    
    return leaf_visited

In [144]:
while not Node.queue_is_empty():

    node = Node.dequeue()
    node.isLeaf = True

In [145]:
total_leaf_node = print_clusters(root, leaf_visited=0)
print(total_leaf_node)

20


In [146]:
label_output = [v for _,v in label.items()]
np.savetxt(output_path, label_output, delimiter=' ', fmt='%i')